In [1]:
# Import Libraries
import os
import ee
import geemap as emap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize() 

Enter verification code: 4/1AfDhmrjlNoO6Cv0X7k_895q2xpwOZVVk8bb1IvBInqiA1kXUQaQqeLUSons

Successfully saved authorization token.


In [48]:
# Initialize Map
add_google_map = False

Map = emap.Map(center=[41.0918,-112.4451], zoom=9)
# Map.add_basemap('SATELLITE') # Add Google Map
Map

Map(center=[41.0918, -112.4451], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleBut…

In [ ]:
# Get PRISM data
dataset = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').filterDate('2020-01-01', '2020-03-31')
precipitation = dataset.select('ppt')

In [53]:
# Calculate mean of selected date range
mean = precipitation.mean()
precipitationVis = {'min': 0.0,
                    'max': 100.0,
                    'palette': ['red', 'yellow', 'green', 'cyan', 'purple']}
Map.addLayer(precipitation, precipitationVis, 'Precipitation')

In [57]:
# Create contours
lines = ee.List.sequence(0, 150, 10)
 
def create_contours(line):
    mycontour = mean.convolve(ee.Kernel.gaussian(5, 3)) \
    .subtract(ee.Image.constant(line)).zeroCrossing() \
    .multiply(ee.Image.constant(line)).toFloat()

    return mycontour.mask(mycontour)

contourlines = lines.map(create_contours)

contourlines = ee.ImageCollection(contourlines).mosaic()
Map.addLayer(contourlines, {'min': 0, 'max': 150, 'palette':['ff0000', '00ff00']}, 'contours')
# Map.addLayer(contourlines, {'min': 0, 'max': 150, 'palette':['black']}, 'contours')
